In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import MissingIndicator,SimpleImputer   

df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [86]:
df=df.drop(columns=['pclass','who','deck','embark_town','alive','sex','parch','sibsp','embarked','class','adult_male','alone'])


In [87]:
df.head()
X=df.drop(columns=['survived'])
y=df['survived']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [88]:
X_train.head()

,age,fare
331,45.5,28.5000
733,23.0,13.0000
382,32.0,7.9250
704,26.0,7.8542
813,6.0,31.2750


In [89]:
si=SimpleImputer()
X_train_trf = si.fit_transform(X_train)
X_test_trf = si.transform(X_test)

In [90]:
X_train_trf

array([[ 45.5   ,  28.5   ],
       [ 23.    ,  13.    ],
       [ 32.    ,   7.925 ],
       ...,
       [ 41.    ,  14.1083],
       [ 14.    , 120.    ],
       [ 21.    ,  77.2875]], shape=(712, 2))

In [91]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
clf.fit(X_train_trf,y_train)

y_pred=clf.predict(X_test_trf)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6480446927374302

In [92]:
mi =MissingIndicator()
mi.fit(X_train)

X_train_missing=mi.transform(X_train)

In [93]:
X_train_missing

X_test_missing=mi.transform(X_test)


In [94]:
X_train['Age_na']=X_train_missing

X_test['Age_na'] = X_test_missing

In [95]:
X_test

,age,fare,Age_na
709,NaN,15.2458,True
439,31.0,10.5000,False
840,20.0,7.9250,False
720,6.0,33.0000,False
39,14.0,11.2417,False
...,...,...,...
433,17.0,7.1250,False
773,NaN,7.2250,True
25,38.0,31.3875,False
84,17.0,10.5000,False


In [96]:

si = SimpleImputer()

X_train_trf2 = si.fit_transform(X_train)
X_test_trf2 = si.transform(X_test)

In [97]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

clf.fit(X_train_trf2,y_train)

y_pred = clf.predict(X_test_trf2)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6368715083798883

In [98]:

si = SimpleImputer(add_indicator=True)
X_train = si.fit_transform(X_train)

X_test = si.transform(X_test)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

clf.fit(X_train_trf2,y_train)

y_pred = clf.predict(X_test_trf2)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6368715083798883

Automatically Select Imputer

In [99]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
 


In [100]:
df=sns.load_dataset('titanic')

df=df.drop(columns=['who','deck','embark_town','alive','embarked','class','adult_male','alone'])
df.head()

,survived,pclass,sex,age,sibsp,parch,fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [101]:
X=df.drop(columns=['survived'])
y=df['survived']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [102]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare
331,1,male,45.5,0,0,28.5000
733,2,male,23.0,0,0,13.0000
382,3,male,32.0,0,0,7.9250
704,3,male,26.0,1,0,7.8542
813,3,female,6.0,4,2,31.2750


In [103]:
numerical_features = ['age','fare']
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

categorical_features =['embarked','sex']
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])

In [104]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ]
)

clf=Pipeline(steps=[('preprocessor',preprocessor),
                    ('classifier',LogisticRegression())
])

In [105]:
from sklearn import set_config
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['embarked', 'sex'])])),
                ('classifier', LogisticRegression())])

In [106]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],  # Numerical imputer
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],  # Categorical imputer
    'classifier__C': [0.1, 1.0, 10, 100]  # Logistic Regression hyperparameter
}
grid_search=GridSearchCV(clf,param_grid,cv=10)
grid_search.fit(X_train,y_train)

grid_search.best_params_

ValueError: 
All the 160 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ayush\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'embarked'

The above exception was th